In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


**Parse all xml files to numeric format**

In [0]:
from bs4 import BeautifulSoup
import os
import csv

folder = '/content/drive/My Drive/KTH_Handtool_Dataset'
subfolder = ['Blue_background']
in_subfolder = ['Artificial', 'Cloudy', 'Directed']
instruments = ['hammer1', 'hammer2', 'hammer3', 'plier1', 'plier2', 'plier3', 'screw1', 'screw2', 'screw3']

with open('train.csv', mode='w') as file:
  for folder_name in subfolder:
    for in_folder_name in in_subfolder:
      for instrument in instruments:
        directory_name = os.path.join(folder, folder_name, 'rgb', in_folder_name, instrument)
        directory_name_xml = os.path.join(folder, folder_name, 'bboxes', in_folder_name, instrument)
        for filename in os.listdir(directory_name_xml):
          label = instrument
          filename_jpg = filename[:-4] + '.jpg'
          filename_str = os.path.join(directory_name, filename_jpg)
          handler = open(os.path.join(directory_name_xml, filename)).read()
          soup = BeautifulSoup(handler, "xml")
          xmin = int(soup.xmin.string)
          xmax = int(soup.xmax.string)
          ymin = int(soup.ymin.string)
          ymax = int(soup.ymax.string)
          row = [filename_str, xmin, ymin, xmax, ymax, label]
          writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
          writer.writerow(row)
csvfile.close()

In [0]:
subfolder = ['Brown_background', 'White_background']
sub_instruments = ['Kinect', 'webcam']
dict_instr = {'Kinect': 'kinect'}

# open file to write to the end of a file
with open('train.csv', mode='a') as file:
  for folder_name in subfolder:
    for in_folder_name in in_subfolder:
      for instrument in instruments:
        dir_name = os.path.join(folder, folder_name, 'rgb', in_folder_name, instrument)
        dir_name_xml = os.path.join(folder, folder_name, 'bbox', in_folder_name, instrument)
        for sub_instrument in sub_instruments:
          directory_name = os.path.join(folder, folder_name, 'rgb', in_folder_name, instrument, sub_instrument)
          if sub_instrument not in os.listdir(dir_name_xml):
            sub_instrument = dict_instr[sub_instrument]
          directory_name_xml = os.path.join(folder, folder_name, 'bbox', in_folder_name, instrument, sub_instrument)
          for filename in os.listdir(directory_name_xml):
            label = instrument
            filename_jpg = filename[:-4] + '.jpg'
            filename_str = os.path.join(directory_name, filename_jpg)
            handler = open(os.path.join(directory_name_xml, filename)).read()
            soup = BeautifulSoup(handler, "xml")
            xmin = int(soup.xmin.string)
            xmax = int(soup.xmax.string)
            ymin = int(soup.ymin.string)
            ymax = int(soup.ymax.string)
            row = [filename_str, xmin, ymin, xmax, ymax, label]
            writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            writer.writerow(row)

In [0]:
/content/drive/My Drive/KTH_Handtool_Dataset/Blue_background/bboxes/Artificial/hammer1/camera1_15.xml